<a href="https://colab.research.google.com/github/pmonda/rpi5-yolo-nn/blob/colab/src/autodidactic_neurosurgeon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
import sys
import os
import torch
import torch.nn as nn
sys.path.append('/content/drive/MyDrive/ML_Research')
from tiny_yolo import *


def split_model(model, split_idx):
    # Use model.cnn instead of model.module_list
    part1 = nn.Sequential(*list(model.cnn.children())[:split_idx])
    return part1

if __name__ == "__main__":
    from PIL import Image
    from utils import *

    model = TinyYoloNet()
    model.float()
    model.load_weights('/content/drive/MyDrive/ML_Research/yolov2-tiny-voc.weights')
    model.eval()

    # Inspect layer list to choose split index
    for idx, layer in enumerate(model.cnn):
        print(f"{idx}: {layer.__class__.__name__}")

    # Choose split point that does NOT include RegionLoss
    split_idx = len(model.cnn)  # All layers up to but not including RegionLoss
    part1 = split_model(model, split_idx)
    part1.cuda()

    # Load and prepare image
    img = Image.open('/content/drive/MyDrive/ML_Research/pmh2mmc4.png').convert('RGB')
    sized = img.resize((416, 416))
    img_tensor = image2torch(sized).cuda()

    # Run inference with feature extractor (part1)
    with torch.no_grad():
        feature_out = part1(img_tensor)

        # Use get_region_boxes to convert feature output to detections
        all_boxes = get_region_boxes(feature_out.data, conf_thresh=0.5, num_classes=20,
                                     anchors=model.anchors, num_anchors=model.num_anchors)
        boxes = nms(all_boxes[0], 0.4)

    # Plot results
    plot_boxes(img, boxes, 'predict_split.jpg', load_class_names('/content/drive/MyDrive/ML_Research/voc.names'))


0: Conv2d
1: BatchNorm2d
2: LeakyReLU
3: MaxPool2d
4: Conv2d
5: BatchNorm2d
6: LeakyReLU
7: MaxPool2d
8: Conv2d
9: BatchNorm2d
10: LeakyReLU
11: MaxPool2d
12: Conv2d
13: BatchNorm2d
14: LeakyReLU
15: MaxPool2d
16: Conv2d
17: BatchNorm2d
18: LeakyReLU
19: MaxPool2d
20: Conv2d
21: BatchNorm2d
22: LeakyReLU
23: MaxPoolStride1
24: Conv2d
25: BatchNorm2d
26: LeakyReLU
27: Conv2d
28: BatchNorm2d
29: LeakyReLU
30: Conv2d
dog: 1.000000
save plot results to predict_split.jpg


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
